In [ ]:
!pip install python-binance



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.6 MB/s eta 0:00:00


In [ ]:
#import os
#os._exit(0)

In [ ]:
import pandas as pd
import asyncio
from binance import AsyncClient
import nest_asyncio
from datetime import datetime

#  nested event loop
nest_asyncio.apply()

async def fetch_historical_data_for_meme_coins(coins):
    API_KEY = "########"
    API_SECRET = "########"

    client = await AsyncClient.create(API_KEY, API_SECRET, testnet=True)

    #  start and end time
    start_time = datetime(datetime.now().year, 1, 17)  #
    end_time = datetime.utcnow()

    data_dict = {}

    for symbol in coins:
        try:
            current_start_time = int(start_time.timestamp() * 1000)
            end_time_ms = int(end_time.timestamp() * 1000)
            all_klines = []

            while current_start_time < end_time_ms:
                # Fetch data with pagination, up to 1000 records per request
                klines = await client.get_historical_klines(
                    symbol, "1h", current_start_time, end_time_ms, limit=1000
                )

                if not klines:
                    break

                all_klines.extend(klines)

                # Update start time to avoid duplicate data
                current_start_time = int(klines[-1][0]) + 1

            if all_klines:

                df = pd.DataFrame(all_klines, columns=[
                    "Open Time", "Open", "High", "Low", "Close", "Volume",
                    "Close Time", "Quote Asset Volume", "Number of Trades",
                    "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore"
                ])

                # Convert timestamps to datetime
                df["Open Time"] = pd.to_datetime(df["Open Time"], unit='ms')
                df["Close Time"] = pd.to_datetime(df["Close Time"], unit='ms')

                df = df[df["Open Time"] >= start_time]

                data_dict[symbol] = df
            else:
                print(f"No data available for {symbol} from the requested start date.")

        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")

    # Save the data to Excel
    file_name = "binance_trumpusdt_data.xlsx"
    with pd.ExcelWriter(file_name) as writer:
        for coin, df in data_dict.items():
            df.to_excel(writer, sheet_name=coin, index=False)

    await client.close_connection()
    print(f"Data saved to {file_name}")


meme_coins = ["TRUMPUSDT"]

# Run the function
loop = asyncio.get_event_loop()
loop.run_until_complete(fetch_historical_data_for_meme_coins(meme_coins))


Data saved to binance_trumpusdt_data.xlsx


In [ ]:
from google.colab import files
files.download("binance_trumpusdt_data.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import requests
import pandas as pd
from datetime import datetime

def fetch_historical_data_kraken(symbol, start_date):
    # Kraken API endpoint
    url = "https://api.kraken.com/0/public/OHLC"

    # Convert start date to UNIX timestamp
    start_timestamp = int(datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S").timestamp())

    params = {
        "pair": symbol,
        "interval": 60,  # 1-hour interval
        "since": start_timestamp
    }

    # Send GET request to Kraken API
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()

        if "result" not in data or symbol not in data["result"]:
            print(f"No data available for {symbol}")
            return None

        ohlc_data = data["result"][symbol]

        # DataFrame with correct column names
        df = pd.DataFrame(ohlc_data, columns=[
            "Time", "Open", "High", "Low", "Close", "Volume", "VWAP", "Trades"
        ])

        df["Time"] = pd.to_datetime(df["Time"], unit="s")

        # Filter data to
        df = df[df["Time"] >= datetime(2025, 1, 18)]

        # additional calculated columns
        df["Price Change"] = df["Close"].astype(float) - df["Open"].astype(float)
        df["Price Change %"] = (df["Price Change"] / df["Open"].astype(float)) * 100
        df["High-Low Spread"] = df["High"].astype(float) - df["Low"].astype(float)
        df["Cumulative Volume"] = df["Volume"].astype(float).cumsum()

        # Add SMA for the last 3 hours
        df["SMA_3"] = df["Close"].astype(float).rolling(window=3).mean()

        # Save to Excel file
        file_name = f"kraken_{symbol}_1h_data.xlsx"
        df.to_excel(file_name, index=False)
        print(f"Data saved to {file_name}")
    else:
        print(f"Failed to fetch data from Kraken. Status code: {response.status_code}")

# Fetch historical data for TRUMPUSDT starting from January 18, 2025
fetch_historical_data_kraken("TRUMPUSDT", "2025-01-18 00:00:00")


Data saved to kraken_TRUMPUSDT_1h_data.xlsx


In [ ]:
from google.colab import files
files.download("kraken_TRUMPUSDT_1h_data.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Data saved to kraken_TRUMPUSDT_4h_data.xlsx
